In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep
import glob

In [6]:
def geocode_df(df, geolocator):
    # Get unique place names
    df['Mandal'] = df['Mandal'].astype(str)
    places = df['Mandal'].unique()
    
    # Define dictionary to store latitudes and longitudes
    lat_longs = {}
    
    # Loop over unique places and geocode
    for place in places:
        # Check if already geocoded
        if place in lat_longs:
            continue
        # Geocode and wait between requests
        location = geolocator.geocode(place + ', Telangana, India')
        if location and location.latitude and location.longitude:
            lat_longs[place] = (location.latitude, location.longitude)
        else:
            lat_longs[place] = (None, None)
        sleep(1)
    
    # Add latitude and longitude columns to DataFrame
    df['latitude'] = df['Mandal'].apply(lambda x: lat_longs[x][0])
    df['longitude'] = df['Mandal'].apply(lambda x: lat_longs[x][1])
    
    # Drop the original "place" column
    # df = df.drop(columns=['place'])
    
    return df

In [7]:
csv_files = glob.glob('*.csv')
dfs = [pd.read_csv(csv_file) for csv_file in csv_files]


In [10]:
geolocator = Nominatim(user_agent='')

In [11]:
for i, df in enumerate(dfs):
    dfs[i] = geocode_df(df, geolocator)

In [ ]:
for i, csv_file in enumerate(csv_files):
    dfs[i].to_csv(csv_file, index=False)